In [2]:
import os

from language_table.environments import blocks
from language_table.environments import language_table
from language_table.environments.rewards import block2block

from matplotlib import pyplot as plt
import mediapy
import numpy as np
import tensorflow_datasets as tfds
from icecream import ic

np.set_printoptions(precision=3)

pybullet build time: Nov 28 2023 23:48:36
2024-06-16 09:26:05.676666: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 09:26:06.803060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# tf dataset handling
import tensorflow as tf

dataset = tf.data.Dataset.range(10)

for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


2024-06-16 09:26:12.517070: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-16 09:26:12.521418: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-16 09:26:12.571713: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

In [4]:
DATASET_PATH = "/data/language_table_sim"
builder = tfds.builder_from_directory(DATASET_PATH)
episode_ds = builder.as_dataset(split="train")
print(episode_ds.element_spec)

{'episode_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'steps': DatasetSpec({'action': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': {'effector_target_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'effector_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'instruction': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'rgb': TensorSpec(shape=(360, 640, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}


In [8]:
type(episode_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [5]:
def decode_inst(inst):
    """Utility to decode encoded language instruction"""
    return bytes(inst[np.where(inst != 0)].tolist()).decode("utf-8")

In [7]:
def show_episode(episode):
    """Utility to visualize an episode"""
    print("Episode id:", episode["episode_id"].numpy().decode("utf-8"))
    frames = []
    actions = []
    for step in episode["steps"].as_numpy_iterator():
        frames.append(step["observation"]["rgb"])
        actions.append(step["action"])

    print("step numbers: ", len(frames))
    mediapy.show_video(
        frames, title=decode_inst(step["observation"]["instruction"]), fps=5
    )
    # draw the action sequence
    actions = np.array(actions)
    actions.min(axis=0)
    print("action sequence:")
    print(actions)
    return actions.min(axis=0), actions.max(axis=0)

In [9]:
episodes_iter = iter(episode_ds.take(5))

# sample_episodes = tf.data.Dataset.sample_from_datasets(episode_ds)
# episodes_iter = iter(sample_episodes)
action_mins = []
action_maxes = []

for episode in episodes_iter:
    action_min, action_max = show_episode(episode)
    action_mins.append(action_min)
    action_maxes.append(action_max)

action_mins = np.array(action_mins)
action_maxes = np.array(action_maxes)

print("global action min:", action_mins.min(axis=0))
print("global action max:", action_maxes.max(axis=0))

Episode id: EiIKGHdvcmtlcl8wODlfZXBfOF8wNl8wNl8yMhAvGDggBDAAKiBmNTI1ODUyODdhMTU3NmI4NWRmYmMyOTliNDk4MTFmYw==
step numbers:  10


2024-06-16 09:48:31.237569: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


surround the red pentagon with the others This browser does not support the video tag.


action sequence:
[[-3.057e-04  1.751e-02]
 [-7.094e-03  9.966e-03]
 [-1.165e-02  2.447e-03]
 [-1.404e-02 -7.375e-05]
 [-7.023e-03 -2.571e-03]
 [-4.685e-03 -2.567e-03]
 [ 8.861e-05 -5.076e-03]
 [-2.658e-04  1.523e-02]
 [-3.987e-04  2.284e-02]
 [ 0.000e+00  0.000e+00]]
Episode id: EiMKGXdvcmtlcl8xMDZfZXBfMTBfMDZfMDZfMjIQORhrIA0wASogZjUyNTg1Mjg3YTE1NzZiODVkZmJjMjk5YjQ5ODExZmM=
step numbers:  51


2024-06-16 09:48:31.542269: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

action sequence:
[[ 0.004 -0.03 ]
 [-0.009 -0.029]
 [-0.009 -0.028]
 [-0.009 -0.029]
 [-0.008 -0.029]
 [-0.015 -0.026]
 [-0.023 -0.019]
 [-0.015  0.003]
 [-0.006  0.018]
 [ 0.002  0.023]
 [ 0.002  0.009]
 [ 0.     0.   ]]
Episode id: EiQKGHdvcmtlcl8xMThfZXBfM18wNV8wOV8yMhCsAxjXAyAJMAQqIGIwMzEwNzBjZjRjZGYwYTJlODgzOWExM2JlZDRkN2Zk
step numbers:  44


2024-06-16 09:48:32.093997: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


put all the blocks in the bottom left corner This browser does not support the video tag.


action sequence:
[[ 1.075e-02 -4.465e-03]
 [ 7.729e-03 -7.696e-03]
 [-5.581e-03 -6.651e-03]
 [-5.782e-03 -1.245e-02]
 [-7.127e-03 -1.401e-02]
 [-4.633e-03 -2.184e-02]
 [-3.035e-03 -1.431e-02]
 [ 2.797e-04 -3.000e-02]
 [-1.765e-03 -2.995e-02]
 [-1.758e-03 -1.486e-02]
 [-2.478e-03  1.897e-03]
 [-1.446e-02  2.628e-02]
 [-1.652e-02  1.898e-02]
 [-6.888e-03  2.712e-03]
 [-5.518e-03 -1.206e-02]
 [ 1.436e-03 -1.450e-02]
 [ 5.464e-03 -1.459e-02]
 [ 7.949e-03 -1.611e-02]
 [ 6.432e-03 -1.219e-02]
 [ 5.037e-03 -8.548e-03]
 [ 4.704e-03 -2.266e-02]
 [-7.749e-05 -4.698e-03]
 [ 1.185e-03 -3.164e-03]
 [ 1.177e-03 -3.144e-03]
 [ 2.424e-03 -1.629e-03]
 [ 2.408e-03 -1.619e-03]
 [ 2.393e-03 -1.609e-03]
 [ 2.378e-03 -1.599e-03]
 [ 3.478e-03 -4.639e-03]
 [ 3.435e-03 -4.581e-03]
 [ 2.313e-03 -1.541e-03]
 [ 1.080e-03 -2.984e-03]
 [-1.527e-02  7.824e-03]
 [-9.735e-03  1.782e-03]
 [-1.251e-02 -1.039e-02]
 [-3.774e-03 -4.497e-03]
 [ 2.467e-03 -6.067e-03]
 [ 4.921e-03 -3.030e-03]
 [ 2.437e-03 -1.501e-03]
 [ 4.829

2024-06-16 09:48:32.272635: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
